# Device

In this notebook you will:

* Encapsulate multiple Signals in a Device

Recommend Prerequisites:

* [Hello Python and Jupyter](./Hello%20Python%20and%20Jupyter.ipynb)
* [Epics Signal](./Epics%20Signal.ipynb)

## Simulated Hardware
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

another_random_walk              RUNNING   pid 4651, uptime 10:02:22
decay                            RUNNING   pid 19219, uptime 0:14:33
mini_beamline                    RUNNING   pid 4704, uptime 14:41:21
random_walk                      RUNNING   pid 4702, uptime 14:41:21
simple                           RUNNING   pid 4703, uptime 14:41:21
thermo_sim                       RUNNING   pid 2106, uptime 10:29:01
trigger_with_pc                  RUNNING   pid 4705, uptime 14:41:21


## Devices are a heirarchy

A `Device` is a hierarchy composed of Signals and other Devices. The components of a Device can be introspected by layers above ophyd and may be decomposed to, ultimately, the underlying Signals.

In [2]:
from ophyd import EpicsSignal, EpicsSignalRO

x = EpicsSignal('random_walk:x')
dt = EpicsSignal('random_walk:dt')

It would be convenient if we could read these as a unit, instead of `x.read(); dt.read()`.

In [3]:
from ophyd import Device, Component

class RandomWalk(Device):
    x = Component(EpicsSignalRO, ':x')
    dt = Component(EpicsSignal, ':dt')
    
random_walk = RandomWalk('random_walk', name='random_walk')
random_walk

RandomWalk(prefix='random_walk', name='random_walk', read_attrs=['x', 'dt'], configuration_attrs=[])

The `read()` and `describe()` methods walk the hierarchy.

In [4]:
random_walk.read()

OrderedDict([('random_walk_x',
              {'value': 5.399934643233662, 'timestamp': 1555322208.884386}),
             ('random_walk_dt',
              {'value': 2.0, 'timestamp': 1555321483.939717})])

In [5]:
random_walk.x.read()

{'random_walk_x': {'value': 5.399934643233662, 'timestamp': 1555322208.884386}}

In [6]:
random_walk.dt.read()

{'random_walk_dt': {'value': 2.0, 'timestamp': 1555321483.939717}}

In [7]:
random_walk.describe()

OrderedDict([('random_walk_x',
              {'source': 'PV:random_walk:x',
               'dtype': 'number',
               'shape': [],
               'units': '',
               'lower_ctrl_limit': 0.0,
               'upper_ctrl_limit': 0.0,
               'precision': 0}),
             ('random_walk_dt',
              {'source': 'PV:random_walk:dt',
               'dtype': 'number',
               'shape': [],
               'units': '',
               'lower_ctrl_limit': 0.0,
               'upper_ctrl_limit': 0.0,
               'precision': 0})])

A Device embodies a certain "layout" of components. We can have multiple Devices with different PV prefixes but the same layout.

In [8]:
another_random_walk = RandomWalk('another_random_walk', name='another_random_walk')

In [9]:
another_random_walk.read()

OrderedDict([('another_random_walk_x',
              {'value': 41.54585527008535, 'timestamp': 1555322208.665025}),
             ('another_random_walk_dt',
              {'value': 3.0, 'timestamp': 1555286063.982111})])

A Device can be made of subdevices.

In [10]:
class RandomWalks(Device):
    a = Component(RandomWalk, 'random_walk')
    b = Component(RandomWalk, 'another_random_walk')
    
random_walks = RandomWalks('', name='random_walks')
random_walks

RandomWalks(prefix='', name='random_walks', read_attrs=['a', 'a.x', 'a.dt', 'b', 'b.x', 'b.dt'], configuration_attrs=['a', 'b'])

In [11]:
random_walks.read()

OrderedDict([('random_walks_a_x',
              {'value': 6.003945764554269, 'timestamp': 1555322210.88516}),
             ('random_walks_a_dt',
              {'value': 2.0, 'timestamp': 1555321483.939717}),
             ('random_walks_b_x',
              {'value': 41.54585527008535, 'timestamp': 1555322208.665025}),
             ('random_walks_b_dt',
              {'value': 3.0, 'timestamp': 1555286063.982111})])

In [12]:
random_walks.a.read()

OrderedDict([('random_walks_a_x',
              {'value': 6.003945764554269, 'timestamp': 1555322210.88516}),
             ('random_walks_a_dt',
              {'value': 2.0, 'timestamp': 1555321483.939717})])

In [13]:
random_walks.a.x.read()

{'random_walks_a_x': {'value': 6.003945764554269,
  'timestamp': 1555322210.88516}}

## Adding a set method to `Device`

Sometimes, setting a value to a Signal and knowing when it is "done" involves just one PV:

In [14]:
status = random_walks.a.dt.set(2)

In other cases it involves coordination across multiple PVs, such as a setpoint PV nd a readback PV, or a setpoint PV and a "done" PV. For those cases, we define a `set` method on the Device to manage the coordination across multiple Signals.

In [22]:
from ophyd import DeviceStatus

class Decay(Device):
    readback = Component(EpicsSignalRO, ':I')
    setpoint = Component(EpicsSignal, ':SP')
    
    def set(self, setpoint):
        """
        Set the setpoint and return a Status object that monitors the readback.
        """
        status = DeviceStatus(self)
        
        # Wire up a callback that will mark the status object as finished
        # when the readback approaches within some tolerance of the setpoint.
        def callback(old_value, value, **kwargs):
            TOLERANCE = 1  # hard-coded; we'll make this configurable later on...
            if abs(value - setpoint) < TOLERANCE:
                status._finished()
            
        self.readback.subscribe(callback)
        
        # Now 'put' the value.
        self.setpoint.put(setpoint)
        
        # And return the Status object, which the caller can use to
        # tell when the action is complete.
        return status
        
    
decay = Decay('decay', name='decay')
decay

Decay(prefix='decay', name='decay', read_attrs=['readback', 'setpoint'], configuration_attrs=[])

In [33]:
decay.read()

OrderedDict([('decay_readback',
              {'value': 120.98177665911605, 'timestamp': 1555322299.327124}),
             ('decay_setpoint',
              {'value': 120.0, 'timestamp': 1555322276.117141})])

In [26]:
status = decay.set(115)

We can watch for completion either by registering a callback:

In [27]:
def callback():
    print("DONE!")
    
status.add_callback(callback)

or by polling:

In [35]:
status = decay.set(120)

import time
while not status.done:
    time.sleep(0.01)  # Make sure to sleep to avoid pinning CPU.
print("DONE!")

DONE!


## Adding a `trigger` method to `Device`

Like `Device.set`, `Device.trigger` can coordinate across multiple PVs to trigger and detector and tell when it is done triggering.

In [ ]:
class TriggeredDetector(Device):
    gain = Component(EpicsSignal, ':gain')
    exposure_time = Component(EpicsSignal, ':exposure_time')
    reading = Component(EpicsSignalRO, ':reading')
    acquire = Component(EpicsSignal, ':acquire')
    enabled = Component(EpicsSignal, ':enabled')

    def trigger(self):
        """
        Trigger the detector and return a Status object.
        """
        status = DeviceStatus(self)
        
        # Wire up a callback that will mark the status object as finished
        # when the readback approaches within some tolerance of the setpoint.
        def callback(old_value, value, **kwargs):
            TOLERANCE = 1  # hard-coded for now...
            if abs(value - setpoint) < TOLERANCE:
                status._finished()
            
        self.readback.subscribe(callback)
        
        # Now 'put' the value.
        self.setpoint.put(setpoint)
        
        # And return the Status object, which the caller can use to
        # tell when the action is complete.
        return status

## Sorting components into "kinds"

## Staging and unstaging

### `stage_sigs`

## Customizing cleanup via `stop`, `resume`, `pause`